<a href="https://colab.research.google.com/github/RenqinSS/Rec/blob/main/algo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import os
import numpy as np
import torch

SEED = 45
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(SEED)

In [ ]:
!git clone https://github.com/dpoqb/wechat_big_data_baseline_pytorch.git

!dir
!mkdir data
!unzip ./drive/MyDrive/wechat_algo_data1.zip -d ./data

!pip install deepctr_torch

import torch
import os

print(torch.cuda.is_available())
for i in range(torch.cuda.device_count()):
    print(torch.cuda.get_device_name(i))

Cloning into 'wechat_big_data_baseline_pytorch'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 16 (delta 3), reused 5 (delta 0), pack-reused 0
Unpacking objects: 100% (16/16), done.
drive  sample_data  wechat_big_data_baseline_pytorch
Archive:  ./drive/MyDrive/wechat_algo_data1.zip
   creating: ./data/wechat_algo_data1/
  inflating: ./data/wechat_algo_data1/test_a.csv  
  inflating: ./data/wechat_algo_data1/feed_info.csv  
  inflating: ./data/wechat_algo_data1/feed_embeddings.csv  
  inflating: ./data/wechat_algo_data1/README.md  
  inflating: ./data/wechat_algo_data1/user_action.csv  
  inflating: ./data/wechat_algo_data1/submit_demo_初赛a.csv  
     |████████████████████████████████| 71kB 7.0MB/s 
True
Tesla V100-SXM2-16GB


In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from sklearn.decomposition import PCA
from collections import defaultdict

import os
os.chdir('/content/wechat_big_data_baseline_pytorch')


# 存储数据的根目录
ROOT_PATH = "../data"
# 比赛数据集路径
DATASET_PATH = ROOT_PATH + '/wechat_algo_data1/'
# 训练集
USER_ACTION = DATASET_PATH + "user_action.csv"
FEED_INFO = DATASET_PATH + "feed_info.csv"
FEED_EMBEDDINGS = DATASET_PATH + "feed_embeddings.csv"
# 测试集
TEST_FILE = DATASET_PATH + "test_a.csv"
# 初赛待预测行为列表
ACTION_LIST = ["read_comment", "like", "click_avatar", "forward"]
FEA_COLUMN_LIST = ["read_comment", "like", "click_avatar", "forward", "comment", "follow", "favorite", "device"]
FEA_FEED_LIST = ['feedid', 'authorid', 'videoplayseconds', 'bgm_song_id', 'bgm_singer_id', 'manual_tag_list']
# 负样本下采样比例(负样本:正样本)
ACTION_SAMPLE_RATE = {"read_comment": 4, "like": 4, "click_avatar": 4, "forward": 10, "comment": 10, "follow": 10, "favorite": 10}

def process_embed(train):
    feed_embed_array = np.zeros((train.shape[0], 512))
    for i in tqdm(range(train.shape[0])):
        x = train.loc[i, 'feed_embedding']
        if x != np.nan and x != '':
            y = [float(i) for i in str(x).strip().split(" ")]
        else:
            y = np.zeros((512,)).tolist()
        feed_embed_array[i] += y
    temp = pd.DataFrame(columns=[f"embed{i}" for i in range(512)], data=feed_embed_array)
    train = pd.concat((train, temp), axis=1)
    return train

def proc_tag(df, name='manual_tag_list', thre=5, max_len=5):
    stat = defaultdict(int)

    for row in df[name]:
        if isinstance(row, str):
            for tag in row.strip().split(';'):
                stat[tag] += 1

    zero_tags = set([tag for tag in stat if stat[tag] < thre])  # 低于频次的 tag

    def tag_func(row, max_len=max_len):
        ret = []
        if isinstance(row, str):
            for tag in row.strip().split(';'):
                ret.append(0 if tag in zero_tags else int(tag) + 1)
        ret = ret[:max_len] + [0] * (max_len - len(ret))
        return ' '.join([str(n) for n in ret])

    df[name] = df[name].apply(tag_func)

    tag_vocab_size = max([int(tag) for tag in stat]) + 2
    print('%s: vocab_size == %d' % (name, tag_vocab_size))
    return df

def prepare_data():
    feed_info_df = pd.read_csv(FEED_INFO)

    feed_info_df = proc_tag(feed_info_df, name='manual_tag_list', thre=5, max_len=5)

    user_action_df = pd.read_csv(USER_ACTION)[["userid", "date_", "feedid",] + FEA_COLUMN_LIST]
    
    feed_info_df = feed_info_df[FEA_FEED_LIST]

    test = pd.read_csv(TEST_FILE)

    # add feed feature
    train = pd.merge(user_action_df, feed_info_df, on='feedid', how='left')
    test = pd.merge(test, feed_info_df, on='feedid', how='left')
    test["videoplayseconds"] = np.log(test["videoplayseconds"] + 1.0)
    test.to_csv(ROOT_PATH + f'/test_data.csv', index=False)
    for action in tqdm(ACTION_LIST):
        print(f"prepare data for {action}")
        tmp = train.drop_duplicates(['userid', 'feedid', action], keep='last')
        df_neg = tmp[tmp[action] == 0]
        df_neg = df_neg.sample(frac=1.0 / ACTION_SAMPLE_RATE[action], random_state=SEED, replace=False)
        df_all = pd.concat([df_neg, tmp[tmp[action] == 1]])
        df_all["videoplayseconds"] = np.log(df_all["videoplayseconds"] + 1.0)
        df_all.to_csv(ROOT_PATH + f'/train_data_for_{action}.csv', index=False)


if __name__ == "__main__":
    prepare_data()

manual_tag_list: vocab_size == 354


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:82: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


prepare data for read_comment
prepare data for like
prepare data for click_avatar
prepare data for forward



In [ ]:
from sklearn.decomposition import PCA

n_dim = 32
feed_embed = pd.read_csv(FEED_EMBEDDINGS)
feed_embed['feed_embedding'] = feed_embed['feed_embedding'].apply(lambda row: [float(x) for x in row.strip().split()])
pca = PCA(n_components=n_dim)
pca_emb = pca.fit_transform(feed_embed['feed_embedding'].tolist())
feed_embed['pca_emb'] = list(pca_emb)
feed_embed = feed_embed[['feedid', 'pca_emb']]
# feed_embed.drop(['feed_embedding'], axis=1).to_csv("/content/drive/MyDrive/pca_emb%d.csv" % n_dim, index=False)

In [ ]:
from numba import njit
from scipy.stats import rankdata


@njit
def _auc(actual, pred_ranks):
    n_pos = np.sum(actual)
    n_neg = len(actual) - n_pos
    return (np.sum(pred_ranks[actual == 1]) - n_pos*(n_pos+1)/2) / (n_pos*n_neg)


def fast_auc(actual, predicted):
    # https://www.kaggle.com/c/riiid-test-answer-prediction/discussion/208031
    pred_ranks = rankdata(predicted)
    return _auc(actual, pred_ranks)


def uAUC(labels, preds, user_id_list):
    user_pred = defaultdict(lambda: [])
    user_truth = defaultdict(lambda: [])
    for idx, truth in enumerate(labels):
        user_id = user_id_list[idx]
        pred = preds[idx]
        truth = labels[idx]
        user_pred[user_id].append(pred)
        user_truth[user_id].append(truth)

    user_flag = defaultdict(lambda: False)
    for user_id in set(user_id_list):
        truths = user_truth[user_id]
        flag = False
        # 若全是正样本或全是负样本，则flag为False
        for i in range(len(truths) - 1):
            if truths[i] != truths[i + 1]:
                flag = True
                break
        user_flag[user_id] = flag

    total_auc = 0.0
    size = 0.0
    for user_id in user_flag:
        if user_flag[user_id]:
            auc = fast_auc(np.asarray(user_truth[user_id]), np.asarray(user_pred[user_id]))
            total_auc += auc 
            size += 1.0
    user_auc = float(total_auc)/size
    return user_auc


def compute_weighted_score(score_dict, weight_dict):
    score = 0.0
    weight_sum = 0.0
    for action in score_dict:
        weight = float(weight_dict[action])
        score += weight*score_dict[action]
        weight_sum += weight
    score /= float(weight_sum)
    score = round(score, 6)
    return score

In [ ]:
sparse_2_dim = {
    'userid': 8,
    'feedid': 8,
    'authorid': 8,
    'bgm_song_id': 8,
    'bgm_singer_id': 8,
}

dense_2_dim = {
    'videoplayseconds': 1,
    'pca_emb': 32,
    #'w2v': 8 * 3
}

var_2_dim = {
    'manual_tag_list': {'dim': 8, 'vocab_size': 354},
}

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from collections import defaultdict
from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names
from deepctr_torch.models.deepfm import *
from deepctr_torch.models.basemodel import *


class MyBaseModel(BaseModel):

    def fit(self, x, y, batch_size, val_data=None, epochs=1, verbose=1, mode='offline'):
        x = [x[feature] for feature in self.feature_index]  # type(x) = dict
        for i in range(len(x)):
            x[i] = np.array(x[i].tolist())
            if len(x[i].shape) == 1:
                x[i] = np.expand_dims(x[i], axis=1)

        val_x, val_y = [], []
        if mode == 'offline':
            val_x, val_y = val_data
            val_uids = val_x['userid'].tolist()
            val_x = [val_x[feature] for feature in self.feature_index]
        
        train_tensor_data = Data.TensorDataset(torch.from_numpy(np.concatenate(x, axis=-1)), torch.from_numpy(y))
        train_loader = DataLoader(dataset=train_tensor_data, shuffle=True, batch_size=batch_size)
        sample_num = len(train_tensor_data)
        steps_per_epoch = (sample_num - 1) // batch_size + 1

        # Train
        print("Train on {0} samples, validate on {1} samples, {2} steps per epoch".format(len(train_tensor_data), len(val_y), steps_per_epoch))
        epoch_logs = defaultdict(dict)
        model = self.train()
        for epoch in range(epochs):
            start_time = time.time()
            loss_epoch = 0
            total_loss_epoch = 0
            train_result = defaultdict(list)
            for _, (x_train, y_train) in tqdm(enumerate(train_loader)):
                x = x_train.to(self.device).float()
                y = y_train.to(self.device).float()

                y_pred = model(x).squeeze()

                self.optim.zero_grad()
                loss = self.loss_func(y_pred, y.squeeze(), reduction='sum')
                total_loss = loss + self.get_regularization_loss() + self.aux_loss

                loss_epoch += loss.item()
                total_loss_epoch += total_loss.item()
                total_loss.backward()
                self.optim.step()

                for name, func in self.metrics.items():
                    try:
                        temp = func(y.cpu().data.numpy(), y_pred.cpu().data.numpy().astype("float64"))
                    except:
                        temp = 0
                    finally:
                        train_result[name].append(temp)

            # Add logs
            logs = {}
            logs["loss"] = total_loss_epoch / sample_num
            for name, result in train_result.items():
                logs[name] = np.sum(result) / steps_per_epoch

            if mode == 'offline':
                eval_result = self.evaluate(val_x, val_y, val_uids, batch_size)
                for name, result in eval_result.items():
                    logs["val_" + name] = result
            
            print('Epoch {0}/{1}, {2}s'.format(epoch + 1, epochs, int(time.time() - start_time)))
            eval_str = "loss: {0: .4f}".format(logs["loss"])
            for name in logs:
                eval_str += " - " + name + ": {0: .4f}".format(logs[name])
            print(eval_str)
            epoch_logs[epoch+1] = logs
        return epoch_logs

    def evaluate(self, x, y, uids, batch_size=256):
        preds = self.predict(x, batch_size)
        eval_result = {}
        for name, metric_fun in self.metrics.items():
            eval_result[name] = metric_fun(y, preds)
        eval_result['uAUC'] = uAUC(y.squeeze(), preds.squeeze(), uids)

        return eval_result

    def predict(self, x, batch_size=256):
        model = self.eval()
        if isinstance(x, dict):
            x = [x[feature] for feature in self.feature_index]
        for i in range(len(x)):
            x[i] = np.array(x[i].tolist())
            if len(x[i].shape) == 1:
                x[i] = np.expand_dims(x[i], axis=1)

        tensor_data = Data.TensorDataset(torch.from_numpy(np.concatenate(x, axis=-1)))
        test_loader = DataLoader(dataset=tensor_data, shuffle=False, batch_size=batch_size)

        pred_ans = []
        with torch.no_grad():
            for _, x_test in enumerate(test_loader):
                x = x_test[0].to(self.device).float()
                y_pred = model(x).cpu().data.numpy()
                pred_ans.append(y_pred)

        return np.concatenate(pred_ans).astype("float64")

class MyDeepFM(MyBaseModel):
    def __init__(self,
                 linear_feature_columns, dnn_feature_columns,
                 dense_map = None, dnn_hidden_units=(256, 128),
                 l2_reg_linear=0.00001, l2_reg_embedding=0.00001, l2_reg_dnn=0, init_std=0.0001, seed=1024,
                 dnn_dropout=0., dnn_activation='relu', dnn_use_bn=True, task='binary', device='cpu'):

        super(MyDeepFM, self).__init__(linear_feature_columns, dnn_feature_columns, l2_reg_linear=l2_reg_linear,
                                     l2_reg_embedding=l2_reg_embedding, init_std=init_std, seed=seed, task=task,
                                     device=device)

        # dense map
        dense_map = {}
        self.dense_map = dense_map
        self.dense_map_dict = dict([(name, nn.Linear(dense_2_dim[name], dense_map[name], bias=False).to(device)) for name in dense_map])
        dim_delta = sum([dense_map[name] - dense_2_dim[name] for name in dense_map])

        # dnn tower
        self.dnn = DNN(self.compute_input_dim(dnn_feature_columns) + dim_delta, dnn_hidden_units,
                        activation=dnn_activation, l2_reg=l2_reg_dnn, dropout_rate=dnn_dropout, use_bn=dnn_use_bn,
                        init_std=init_std, seed=seed, device=device)
        self.dnn_linear = nn.Linear(dnn_hidden_units[-1], 1, bias=False).to(device)
        self.add_regularization_weight(filter(lambda x: 'weight' in x[0] and 'bn' not in x[0], self.dnn.named_parameters()), l2=l2_reg_dnn)
        self.add_regularization_weight(self.dnn_linear.weight, l2=l2_reg_dnn)

        self.to(device)

    def forward(self, X):
        sparse_embedding_list, dense_value_list = self.input_from_feature_columns(X, self.dnn_feature_columns, self.embedding_dict) # 5*[512,1,4], 1*[512,1]
        
        # lr
        logit = self.linear_model(X)
        
        # fm
        fm_input = torch.cat(sparse_embedding_list, dim=1)
        square_of_sum = torch.pow(torch.sum(fm_input, dim=1, keepdim=True), 2)
        sum_of_square = torch.sum(fm_input * fm_input, dim=1, keepdim=True)
        logit += 0.5 * torch.sum(square_of_sum - sum_of_square, dim=2, keepdim=False)

        # dense map
        dense_names = [fc.name for fc in self.dnn_feature_columns if isinstance(fc, DenseFeat)]
        tmp = []
        for name, tensor in zip (dense_names, dense_value_list):
            if name in self.dense_map_dict:
                tensor = self.dense_map_dict[name](tensor)
            tmp.append(tensor)
        dense_value_list = tmp

        # dnn tower
        sparse_dnn_input = torch.flatten(torch.cat(sparse_embedding_list, dim=-1), start_dim=1)
        dense_dnn_input = torch.flatten(torch.cat(dense_value_list, dim=-1), start_dim=1)
        dnn_input = torch.cat([sparse_dnn_input, dense_dnn_input], dim=-1)
        logit += self.dnn_linear(self.dnn(dnn_input))
        
        return self.out(logit)


mode = 'online'  # online
if __name__ == "__main__":
    submit = pd.read_csv(ROOT_PATH + '/test_data.csv')[['userid', 'feedid']]
    logs = {}
    for action in ACTION_LIST:
        print('*** train for %s ***' % action)

        USE_FEAT = ['userid', 'feedid', 'device', action] + FEA_FEED_LIST[1:]
        train = pd.read_csv(ROOT_PATH + f'/train_data_for_{action}.csv')[['date_'] + USE_FEAT]

        # TODO: sampling
        # train = train.sample(frac=0.1, random_state=42).reset_index(drop=True)
        print("positive ratio:", sum((train[action] == 1) * 1) / train.shape[0])
        
        test = pd.read_csv(ROOT_PATH + '/test_data.csv')[[i for i in USE_FEAT if i != action]]
        test[action] = 0
        test['date_'] = 15
        test = test[['date_'] + USE_FEAT]
        data = pd.concat((train, test)).reset_index(drop=True)

        # universal embedding
        data = pd.merge(data, feed_embed, on='feedid', how='left')
        data['pca_emb'] = [e if isinstance(e, np.ndarray) else np.zeros((32)) for e in data['pca_emb']]
        data['manual_tag_list'] = data['manual_tag_list'].apply(lambda row: np.array([int(x) for x in row.split()]))

        # features
        sparse_features = list(sparse_2_dim.keys())
        dense_features = list(dense_2_dim.keys())
        var_features = list(var_2_dim.keys())
        print('sparse_features: ', sparse_features)
        print('dense_features: ', dense_features)
        print('var_features: ', var_features)

        data[sparse_features] = data[sparse_features].fillna(0)
        data[dense_features] = data[dense_features].fillna(0)

        # 1.Label Encoding for sparse features,and do simple Transformation for dense features
        for feat in sparse_features:
            lbe = LabelEncoder()
            data[feat] = lbe.fit_transform(data[feat])
        # mms = MinMaxScaler(feature_range=(0, 1))
        # data[dense_features] = mms.fit_transform(data[dense_features])

        # 2.count #unique features for each sparse field,and record dense feature field name
        varlen_feature_columns = [VarLenSparseFeat(SparseFeat(feat, vocabulary_size=var_2_dim[feat]['vocab_size'], embedding_dim=var_2_dim[feat]['dim']), maxlen=5, combiner='sum') for feat in var_features]
        
        fixlen_feature_columns = [SparseFeat(feat, data[feat].nunique(), sparse_2_dim[feat]) for feat in sparse_features] + [DenseFeat(feat, dense_2_dim[feat]) for feat in dense_features]
        dnn_feature_columns = fixlen_feature_columns + varlen_feature_columns
        linear_feature_columns = fixlen_feature_columns + varlen_feature_columns

        feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

        # 3.generate input data for model
        train, test = data.iloc[:train.shape[0]].reset_index(drop=True), data.iloc[train.shape[0]:].reset_index(drop=True)
        if mode == 'offline':
            train_idxes, eval_idxes = train['date_'] != 14, train['date_'] == 14
            train, eval = train[train_idxes].drop(['date_'], axis=1), train[eval_idxes].drop(['date_'], axis=1)
        if mode == 'online':
            train = train.drop(['date_'], axis=1)
            eval = train.head()  # fake
        test = test.drop(['date_'], axis=1)

        train_x = {name: train[name] for name in feature_names}
        eval_x  = {name: eval[name]  for name in feature_names}
        test_x  = {name: test[name]  for name in feature_names}

        # 4.Define Model,train,predict and evaluate
        model = MyDeepFM(
            linear_feature_columns=linear_feature_columns, 
            dnn_feature_columns=dnn_feature_columns,
            task='binary', l2_reg_embedding=1e-1, device='cuda:0' if torch.cuda.is_available() else 'cpu', seed=SEED)
        model.compile("adagrad", "binary_crossentropy", metrics=["binary_crossentropy", "auc"])
        
        act_logs = model.fit(train_x, train[[action]].values, val_data=(eval_x, eval[[action]].values), batch_size=512, epochs=2, mode=mode)
        logs[action] = act_logs

        # online
        submit[action] = model.predict(test_x, 128)
        torch.cuda.empty_cache()
    
    # weighted uAUC
    if mode == 'offline':
        score_dict = {}
        for act in logs:
            act_logs = logs[act]
            score_dict[act] = act_logs[max(act_logs.keys())]['val_uAUC']
        weight_dict = {"read_comment": 4.0, "like": 3.0, "click_avatar": 2.0, "forward": 1.0, "favorite": 1.0, "comment": 1.0, "follow": 1.0}
        weighted_uAUC = compute_weighted_score(score_dict, weight_dict)
        print(score_dict)
        print('weighted_uAUC: ', weighted_uAUC)

    # online
    submit.to_csv("./submit_2_45.csv", index=False)


In [ ]:
todo:
  不同的action使用不同的epoch
  seed

NameError: ignored

In [ ]:
int(k[1:-1].strip().split(',')[1])

7

In [ ]:
p = data['manual_tag_list'].apply(lambda row: np.array([int(x) for x in row.split()]))

In [ ]:
p[0].dtype

dtype('int64')

In [ ]:
# baseline
{'read_comment': 0.6102415130979689, 'like': 0.6055234369612766, 'click_avatar': 0.7059927976309249, 'forward': 0.6832353813536607}
weighted_uAUC:  0.635276

# dnn_dropout = 0.1
{'read_comment': 0.6094100217906185, 'like': 0.6052801328988395, 'click_avatar': 0.7059140934189055, 'forward': 0.6846734262464789}
weighted_uAUC:  0.634998

# 256, 128, 128
{'read_comment': 0.613116787160124, 'like': 0.6062583852548347, 'click_avatar': 0.7058735217580193, 'forward': 0.6769030704770939}
weighted_uAUC:  0.635989

# epoch = 2
{'read_comment': 0.6117841889858322, 'like': 0.6089919743022709, 'click_avatar': 0.7138421964649098, 'forward': 0.6829949302549756}
weighted_uAUC:  0.638479

# sparse dim = 8, epoch = 2 (new baseline)
{'read_comment': 0.6126884118803656, 'like': 0.6078158393185238, 'click_avatar': 0.7141126528216767, 'forward': 0.6923154125787877}
weighted_uAUC:  0.639474

# 删除了对 videoplayseconds 的归一化(new baseline)
{'read_comment': 0.6150373746448982, 'like': 0.6087792274162345, 'click_avatar': 0.7137088800810096, 'forward': 0.6919173648006157}
weighted_uAUC:  0.640582

# add feed embedding 32(new baseline)
{'read_comment': 0.6231230935993682, 'like': 0.6162679088683002, 'click_avatar': 0.7128391281987229, 'forward': 0.6951917541544708}
weighted_uAUC:  0.646217

# add feed embedding 64
{'read_comment': 0.6179610910963779, 'like': 0.617180918593666, 'click_avatar': 0.7121687727167492, 'forward': 0.6969728833664359}
weighted_uAUC:  0.64447

# sparse dim = 12
{'read_comment': 0.6152862366363533, 'like': 0.6172504324924313, 'click_avatar': 0.7100718453099804, 'forward': 0.701999472669805}
weighted_uAUC:  0.643504

# baseline 的重复实验，线上 0.656674
{'read_comment': 0.6220072250372239, 'like': 0.6181791275945606, 'click_avatar': 0.7129768375663601, 'forward': 0.6987107057431032}
weighted_uAUC:  0.646723

# (256, 128, 64)
{'read_comment': 0.6176483873668901, 'like': 0.6170515088013665, 'click_avatar': 0.713929279701119, 'forward': 0.6961728898267605}
weighted_uAUC:  0.644578

# dnn_use_bn = True(new baseline) 线上 0.65576
{'read_comment': 0.6269150887735059, 'like': 0.6245276506750953, 'click_avatar': 0.715901103365852, 'forward': 0.7038550482328185}
weighted_uAUC:  0.65169

# dropout = 0.1
{'read_comment': 0.6261901395330384, 'like': 0.6239817428964435, 'click_avatar': 0.7163355996839406, 'forward': 0.6963938852580808}
weighted_uAUC:  0.650577

# baseline 重复实验
{'read_comment': 0.6296090935187716, 'like': 0.621418106767897, 'click_avatar': 0.7168717294762987, 'forward': 0.6967281458462133}
weighted_uAUC:  0.651316

# 先使用linear对ue降维（32->16），再接入dnn
{'read_comment': 0.6227016241604177, 'like': 0.6199456756568217, 'click_avatar': 0.7143073328988507, 'forward': 0.6791432406166807}
weighted_uAUC:  0.64584

# ue(32->4) dnn_use_bn = True
{'read_comment': 0.6214967487996874, 'like': 0.6117349933619828, 'click_avatar': 0.7123781829253133, 'forward': 0.6907462327570015}
weighted_uAUC:  0.643669

# ue(32->8) dnn_use_bn = True
{'read_comment': 0.6255372000774586, 'like': 0.6099148843168334, 'click_avatar': 0.7147080055545442, 'forward': 0.6913280305289646}
weighted_uAUC:  0.645264

# ue(32->16) dnn_use_bn = True
{'read_comment': 0.6230882216710302, 'like': 0.620136770671566, 'click_avatar': 0.716609921279133, 'forward': 0.6855595234090964}
weighted_uAUC:  0.647154

# ue(32->32) dnn_use_bn = True
{'read_comment': 0.6244980658541014, 'like': 0.6178982426111442, 'click_avatar': 0.7149869209063016, 'forward': 0.7032611484183776}
weighted_uAUC:  0.648492

# ue(32) dnn_use_bn = True * 2
{'read_comment': 0.6268910782755379, 'like': 0.6222017679020581, 'click_avatar': 0.7150488812479852, 'forward': 0.6991933553474539}
weighted_uAUC:  0.650346

# ue(32) dnn_use_bn = True 换gpu跑
{'read_comment': 0.6244744632801036, 'like': 0.6220399203865046, 'click_avatar': 0.7144416351024233, 'forward': 0.6968944127096272}
weighted_uAUC:  0.64898

# ue(32) dnn_use_bn = True 换gpu跑
{'read_comment': 0.6275061489685031, 'like': 0.6229652888075203, 'click_avatar': 0.7143199406719899, 'forward': 0.6981564617860107}
weighted_uAUC:  0.650572

# ue(32) dnn_use_bn = False
{'read_comment': 0.621629670929673, 'like': 0.6172394906951797, 'click_avatar': 0.7138702099995802, 'forward': 0.6971143776259561}
weighted_uAUC:  0.646309

# ue(32->32) dnn_use_bn = False
{'read_comment': 0.6185276618440073, 'like': 0.6170842463943023, 'click_avatar': 0.7135048353197133, 'forward': 0.6971910507462337}
weighted_uAUC:  0.644956

# 采样 4 4 4 10 (new baseline)
{'read_comment': 0.6313212962172534, 'like': 0.6220075992585066, 'click_avatar': 0.7143381748417214, 'forward': 0.6978311251747286}
weighted_uAUC:  0.651782

# + device
{'read_comment': 0.6244486461941755, 'like': 0.6263065400087143, 'click_avatar': 0.7067274451536654, 'forward': 0.7078156246005303}
weighted_uAUC:  0.649798

{'read_comment': 0.6239386227276458, 'like': 0.6246523621908081, 'click_avatar': 0.7127589873646121, 'forward': 0.7021687370818925}
weighted_uAUC:  0.64974

# baseline
{'read_comment': 0.6288471047258013, 'like': 0.6219350043589409, 'click_avatar': 0.7146902582599014, 'forward': 0.6918359172388889}
weighted_uAUC:  0.650241

# seed = 80 split_seed = 42
{'read_comment': 0.6242634094879379, 'like': 0.6264243440165063, 'click_avatar': 0.7118413185387293, 'forward': 0.6945659928938664}
weighted_uAUC:  0.649458

# seed = 80
{'read_comment': 0.6204573895103794, 'like': 0.62392994828236, 'click_avatar': 0.7235837537540227, 'forward': 0.6789320461307407}
weighted_uAUC:  0.647972

seed = 81
{'read_comment': 0.6224667184141309, 'like': 0.6232470714913648, 'click_avatar': 0.710962678982845, 'forward': 0.6922917821851972}
weighted_uAUC:  0.647383

# seed 41 42 43 44 45 avg online
{'read_comment': 0.644098, 'like': 0.63073, 'click_avatar': 0.733325, 'forward': 0.697216}
weighted_uAUC:  0.663245


# seed = 41 ; with manual_tags_dim = 8
{'read_comment': 0.6248747741666473, 'like': 0.6202584290770113, 'click_avatar': 0.7195684676287956, 'forward': 0.695015289241745}
weighted_uAUC:  0.649443

### 换gpu ###
# seed = 42 ; with manual_tags_dim = 8
{'read_comment': 0.6326313492611053, 'like': 0.6238611589799947, 'click_avatar': 0.7190238030791103, 'forward': 0.7043097908858788}
weighted_uAUC:  0.654447

# 线上 42
{'read_comment': 0.636948, 'like': 0.623838, 'click_avatar': 0.727661, 'forward': 0.693742}
weighted_uAUC:  0.656837

# 线上 seed 41 42 43 44 45 avg
{'read_comment': 0.646524, 'like': 0.629584, 'click_avatar': 0.733393, 'forward': 0.698964}
weighted_uAUC:  0.66406

# seed = 42 ; w/o manual_tags_dim = 8
{'read_comment': 0.6227764240658794, 'like': 0.6238680596273618, 'click_avatar': 0.7145209265367466, 'forward': 0.7000066937625322}
weighted_uAUC:  0.649176

In [ ]:
data.head()

In [ ]:
USE_FEAT